# Modelización Supervisada

El objetivo es predecir la masa resultante de la colision entre dos electrones. Para darle una capa extra al proyecto, se prueban diferentes modelos de ML a los cuales se les aplican los metodos de optimizaciones GridSearchCV y RandomizedSearchCV de manera simultanea. Asi se pueden comparar los resultados, la cantidad de hiperparametros necesaria para generarlo y el tiempo que tomo el modelo bajo cada metodo. 

## 1. Importar paquetes

In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import time
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRFRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

## 2. Carga de datos

In [3]:
ruta = 'C:/Users/matia/OneDrive/Escritorio/Mass_Prediction'

### 2.1 Datos de entrenamiento

In [4]:
datos = 'trabajo.csv'
ruta_completa = ruta + '/02_Datos/03_Trabajo/' + datos
df_train = pd.read_csv(ruta_completa, index_col = 0, delimiter = ',')
df_train.head()

,Run,Event,E1,px1,py1,pz1,pt1,eta1,phi1,Q1,E2,px2,py2,pz2,pt2,eta2,phi2,Q2,M
0,147115,366639895,58.71410,-7.31132,10.531000,-57.29740,12.82020,-2.202670,2.177660,1,11.28360,-1.03234,-1.88066,-11.07780,2.14537,-2.344030,-2.07281,-1,8.94841
1,147115,366704169,6.61188,-4.15213,-0.579855,-5.11278,4.19242,-1.028420,-3.002840,-1,17.14920,-11.71350,5.04474,11.46470,12.75360,0.808077,2.73492,1,15.89300
5,147115,366663412,6.39616,-5.45672,-2.090680,-2.60078,5.84352,-0.431551,-2.775710,-1,21.38650,15.16980,-8.87030,-12.18930,17.57280,-0.647450,-0.52912,-1,18.40230
7,147115,367133576,77.00570,10.00290,9.175450,-75.80000,13.57370,-2.421030,0.742282,1,9.11623,-1.72295,-1.48674,-8.82761,2.27574,-2.064940,-2.42965,1,11.29120
9,147115,367825395,27.88120,11.93900,-18.346200,17.26960,21.88880,0.724032,-0.993887,1,12.92180,-5.02630,11.60260,2.66263,12.64450,0.209050,1.97960,-1,34.26850


### 2.2 Datos de validación

In [5]:
validacion = 'validacion.csv'
ruta_completa = ruta + '/02_Datos/02_Validacion/' + validacion
df_val = pd.read_csv(ruta_completa, index_col = 0, delimiter = ',')
df_val.head()

,Run,Event,E1,px1,py1,pz1,pt1,eta1,phi1,Q1,E2,px2,py2,pz2,pt2,eta2,phi2,Q2,M
42,147115,370572917,54.50060,-5.61591,17.703900,51.23820,18.57330,1.739250,1.877970,1,3.57962,0.487619,-3.540280,0.205706,3.57371,0.057529,-1.43392,-1,22.35920
57838,148031,32314239,25.31320,5.44066,23.812500,-6.64226,24.42620,-0.268687,1.346170,1,12.61800,0.298286,-3.838940,12.016100,3.85051,1.855930,-1.49325,-1,31.27320
83060,147926,356834649,20.17700,7.36378,-11.803200,-14.61400,13.91190,-0.916611,-1.013000,1,8.24643,-3.648180,-1.509070,-7.239970,3.94797,-1.366760,-2.74937,-1,11.80130
18806,149181,997167820,20.55200,9.88102,2.629740,-17.82790,10.22500,-1.322700,0.260111,1,6.61350,-1.333330,-0.609515,-6.448960,1.46604,-2.187180,-2.71282,-1,8.45300
63365,147754,34538713,9.47001,-2.63765,-0.397099,9.08659,2.66738,1.939730,-2.992160,-1,69.20350,-5.714520,16.111700,67.058700,17.09510,2.075790,1.91164,1,8.64253


## 3. Preprocesamiento

### 3.1 Limpiar espacios variables

In [6]:
df_train.columns = df_train.columns.str.strip()
df_val.columns = df_val.columns.str.strip()

### 3.2 Selección de variables

In [7]:
cols = ['E1', 'E2', 'px1', 'py1', 'pz1', 'px2', 'py2', 'pz2', 
            'pt1', 'pt2', 'eta1', 'eta2', 'phi1', 'phi2', 'Q1', 'Q2', 'M']

### 3.3 Duplicados

In [8]:
df_train = df_train.drop_duplicates()
df_val = df_val.drop_duplicates()

### 3.4 Pipeline de preprocesamiento

In [9]:
preprocesamiento_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Imputar nulos con la media
    ('scaler', StandardScaler())  # Normalizar los datos
])

Aplicación de la pipeline

In [10]:
df_train[cols] = preprocesamiento_pipeline.fit_transform(df_train[cols])
df_val[cols] = preprocesamiento_pipeline.transform(df_val[cols])

## 4. Separación de los set en entrenamiento y target

In [11]:
train_x = df_train.drop(columns = 'M')
train_y = df_train['M']
val_x = df_val.drop(columns = 'M')
val_y = df_val['M']

## 5. Metricas

Instanciar las metricas de forma previa, dado que se van a utilizar las mismas en todos los modelos.

In [12]:
metricas = {
    'MAE': mean_absolute_error,
    'MSE': mean_squared_error,
    'R2': r2_score
}

## 6. Modelización

### 5.1 Regresión lineal

Instanciando el modelo

In [11]:
model = LinearRegression()

Definición de los hiperparametros a probar

In [12]:
grid = {
    'fit_intercept': [True, False],
    'copy_X' : [True, False],
    'positive': [True, False]
}

Entrenamiento utilizando *GridSearch*

In [13]:
start_time = time.time()
grid_search = GridSearchCV(model, grid, cv = 5, scoring = 'r2', n_jobs = -1)
grid_search.fit(train_x, train_y)
grid_time = time.time() - start_time

Recuperar la mejor configuración

In [47]:
best_model_grid = grid_search.best_estimator_
best_model_grid

LinearRegression(fit_intercept=False, positive=True)

Entrenaminto utilizando *RandomizedSearch*

In [48]:
start_time = time.time()
random_search = RandomizedSearchCV(model, param_distributions = grid, n_iter = 2, cv = 5, scoring = 'r2', n_jobs = -1, random_state = 42)
random_search.fit(train_x, train_y)
random_time = time.time() -start_time

Recuperar la mejor configuración

In [49]:
best_model_random = random_search.best_estimator_
best_model_random

LinearRegression()

Utilizar las mejores configuración sobre los datos del set de validación

In [50]:
y_pred_grid = best_model_grid.predict(val_x)
y_pred_random = best_model_random.predict(val_x)

**Resultados de GridSearch**

In [52]:
print('\n Resultados Grid Search')
print(f'Tiempo de búsqueda: {grid_time: .2f} segundos')
for name, metrics in metricas.items():
    print(f'{name}: {metrics(val_y, y_pred_grid): .4f}')


 Resultados Grid Search
Tiempo de búsqueda:  3.62 segundos
MAE:  0.5632
MSE:  0.5954
R2:  0.4096


**Resultados de RandomizedSearch**

In [53]:
print('\n Resultados Random Search')
print(f'Tiempo de búsqueda: {random_time: .2f} segundos')
for name, metrics in metricas.items():
    print(f'{name}: {metrics(val_y, y_pred_random): .4f}')


 Resultados Random Search
Tiempo de búsqueda:  1.15 segundos
MAE:  0.5632
MSE:  0.5953
R2:  0.4097


**Conclusiones iniciales:** La diferencia significativa esta en el tiempo de computo, donde *RandomSearch* utilizo aproximadamente un tiempo menor que *GridSearch* y obtuvo el mismo rendimiento bajo las mismas metricas.

### 5.2 Ridge

Instanciar el modelo

In [21]:
model = Ridge()

Definir los hiperparametros a probar

In [24]:
grid = {
    'alpha': [0.1, 1, 10, 100, 1000],
    'fit_intercept': [True, False]
}

Entrenamiento utilizando *GridSearchCV*

In [37]:
start_time = time.time()
grid_search = GridSearchCV(model, grid, cv = 5, scoring = 'r2', n_jobs = -1)
grid_search.fit(train_x, train_y)
grid_time = time.time() - start_time

C:\Users\matia\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=1.07156e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Recuperar la mejor configuración

In [38]:
best_model_grid = grid_search.best_estimator_
best_model_grid

Ridge(alpha=100)

Entrenar con *RandomizedSearchCV*

In [36]:
start_time = time.time()
random_search = RandomizedSearchCV(model, param_distributions = grid, n_iter = 5, cv = 5, scoring = 'r2', n_jobs = -1, random_state = 42)
random_search.fit(train_x, train_y)
random_time = time.time() - start_time

C:\Users\matia\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:204: LinAlgWarning: Ill-conditioned matrix (rcond=1.06338e-18): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Recuperar la mejor configuración

In [39]:
best_model_random = random_search.best_estimator_
best_model_random

Ridge(alpha=0.1)

Utilizar las mejores configuraciones sobre los datos de validación

In [40]:
y_pred_grid = best_model_grid.predict(val_x)
y_pred_random = best_model_random.predict(val_x)

**Resultados de GridSearch**

In [42]:
print('\n Resultados Grid Search')
print(f'Tiempo de búsqueda: {grid_time: .2f} segundos')
for name, metrics in metricas.items():
    print(f'{name}: {metrics(val_y, y_pred_grid): .4f}')


 Resultados Grid Search
Tiempo de búsqueda:  3.18 segundos
MAE:  0.5634
MSE:  0.5953
R2:  0.4097


**Resultados de RandomizedSearch**

In [43]:
print('\n Resultados Grid Search')
print(f'Tiempo de búsqueda: {random_time: .2f} segundos')
for name, metrics in metricas.items():
    print(f'{name}: {metrics(val_y, y_pred_random): .4f}')


 Resultados Grid Search
Tiempo de búsqueda:  2.34 segundos
MAE:  0.5632
MSE:  0.5953
R2:  0.4097


**Conclusiones iniciales:** No hay mucha diferencia en el rendimiento de ambos resultados.

### 5.3 DecisionTreeRegressor

Instanciar el modelo

In [11]:
model = DecisionTreeRegressor(random_state = 42)

Definir los hiperparametros a probar

In [12]:
grid = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

Entrenar con *GridSearchCV*

In [14]:
start_time = time.time()
grid_search = GridSearchCV(model, grid, cv = 5, scoring = 'r2', n_jobs = -1)
grid_search.fit(train_x, train_y)
grid_time = time.time() - start_time

Recuperar la mejor configuración

In [16]:
best_model_grid = grid_search.best_estimator_
best_model_grid

DecisionTreeRegressor(min_samples_leaf=4, min_samples_split=10, random_state=42)

Entrenar con *RandomizedSearchCV*

In [15]:
start_time = time.time()
random_search = RandomizedSearchCV(model, param_distributions = grid, n_iter = 5, cv = 5, scoring = 'r2', n_jobs = -1, random_state = 42)
random_search.fit(train_x, train_y)
random_time = time.time() - start_time

Recuperar la mejor configuración

In [17]:
best_model_random = random_search.best_estimator_
best_model_random

DecisionTreeRegressor(min_samples_leaf=4, min_samples_split=10, random_state=42)

Utilizar las mejores configuraciones sobre los datos de validación

In [18]:
y_pred_grid = best_model_grid.predict(val_x)
y_pred_random = best_model_random.predict(val_x)

**Resultados de GridSearch**

In [21]:
print('\n Resultados del GridSearch')
print(f'Tiempo de búsqueda: {grid_time: .2f} segundos')
for name, metric in metricas.items():
    print(f'{name}: {metric(val_y, y_pred_grid): .4f}')


 Resultados del GridSearch
Tiempo de búsqueda:  77.08 segundos
MAE:  0.2465
MSE:  0.1894
R2:  0.8122


**Resultados de RandomizedSearch**

In [22]:
print('\n Resultados del Random Search')
print(f'Tiempo de búsqueda: {random_time: .2f} segundos')
for name, metric in metricas.items():
    print(f'{name}: {metric(val_y, y_pred_random): .4f}')


 Resultados del Random Search
Tiempo de búsqueda:  12.29 segundos
MAE:  0.2465
MSE:  0.1894
R2:  0.8122


**Conclusiones iniciales**: ambos metodos llegaron a la misma parametrización del modelo, en conjunto con los buenos resultados, lo que puede ser un indicador de que el modelo es efectivo para este problema. La diferenciaa destacar este en el tiempo que toma *RandomizedSearchCV* siendo más rapido para determinar la mejor configuración.

### 5.4 RandomForestRegressor

Instanciar el modelo

In [14]:
model = RandomForestRegressor(random_state = 42, n_jobs = -1)

Los parametros para el *GridSearch* se reducen a solo uno dado que:

- El algoritmo tomaba mucho tiempo en entrenar
- Al ver los resultados del *RandomizedSearch* el parametro *n_estimators* se presentaba como el más relavante

Tambien se redujo el valor de *cv* para mejorar el rendimiento del entrenamiento.

In [16]:
grid_search = {
    'n_estimators': [50, 100, 200]
}

Los hiperpametros a probar para *RandomizedSearchCV* son los siguientes

In [17]:
grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

Entrenar bajo *GridSearch*

In [18]:
start_time = time.time()
grid_search = GridSearchCV(model, grid_search, cv = 3, scoring = 'r2', n_jobs = -1)
grid_search.fit(train_x, train_y)
grid_time = time.time() - start_time

Recuperar la mejor configuración

In [20]:
best_model_grid = grid_search.best_estimator_
best_model_grid

RandomForestRegressor(n_estimators=200, n_jobs=-1, random_state=42)

Entrenar bajo *RandomizedSearch*

In [19]:
start_time = time.time()
random_search = RandomizedSearchCV(model, param_distributions = grid, n_iter = 5, cv = 5, scoring = 'r2', n_jobs = -1, random_state = 42)
random_search.fit(train_x, train_y)
random_time = time.time() - start_time

Recuperar la mejor configuración

In [21]:
best_model_random = random_search.best_estimator_
best_model_random

RandomForestRegressor(n_estimators=50, n_jobs=-1, random_state=42)

Utilizar las mejores configuraciones sobres los datos de validación

In [22]:
y_pred_grid = best_model_grid.predict(val_x)
y_pred_random = best_model_random.predict(val_x)

**Resultados de GridSearch**

In [23]:
print('\n Resultados del GridSearch')
print(f'Tiempo de búsqueda: {grid_time: .2f} segundos')
for name, metric in metricas.items():
    print(f'{name}: {metric(val_y, y_pred_grid): .4f}')


 Resultados del GridSearch
Tiempo de búsqueda:  578.09 segundos
MAE:  0.1679
MSE:  0.0612
R2:  0.9394


**Resultados de RandomizedSearch**

In [24]:
print('\n Resultados del Random Search')
print(f'Tiempo de búsqueda: {random_time: .2f} segundos')
for name, metric in metricas.items():
    print(f'{name}: {metric(val_y, y_pred_random): .4f}')


 Resultados del Random Search
Tiempo de búsqueda:  680.20 segundos
MAE:  0.1723
MSE:  0.0640
R2:  0.9365


**Conclusiones iniciales:** ambos metodos muestran buenos resultados de acuerdo con las metricas, además de optimizar el mismo hiperparametros. La diferencia de tiempo es significantiva considerando que *RandomizedSearchCV* le tomo 100 segundos más en obtener resultados.

### 5.5 GradientBoostingRegressor

Instanciar el modelo

In [13]:
model = GradientBoostingRegressor(random_state = 42)

Definir los hiperparametros a probar

In [14]:
grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 10],
    'subsample': [0.8, 1.0]
}

Entrenar bajo *GridSearchCV*

In [15]:
start_time = time.time()
grid_search = GridSearchCV(model, grid, cv= 3, scoring = 'r2', n_jobs = -1)
grid_search.fit(train_x, train_y)
grid_time = time.time() - start_time

Recuperar la mejor configuración

In [17]:
best_model_grid = grid_search.best_estimator_
best_model_grid

GradientBoostingRegressor(learning_rate=0.2, max_depth=5, n_estimators=200,
                          random_state=42, subsample=0.8)

Entrenar bajo *RandomizedSearchCV*

In [16]:
start_time = time.time()
random_search = RandomizedSearchCV(model, param_distributions = grid, n_iter = 5, cv = 5, scoring = 'r2', n_jobs = -1, random_state = 42)
random_search.fit(train_x, train_y)
random_time = time.time() - start_time

Recuperar la mejor configuración

In [18]:
best_model_random = random_search.best_estimator_
best_model_random

GradientBoostingRegressor(learning_rate=0.2, max_depth=5, random_state=42,
                          subsample=0.8)

Utilizar las mejores configuraciones sobre los datos de validación

In [19]:
y_pred_grid = best_model_grid.predict(val_x)
y_pred_random = best_model_random.predict(val_x)

**Resultados de GridSearch**

In [20]:
print('\n Resultados del GridSearch')
print(f'Tiempo de búsqueda: {grid_time: .2f} segundos')
for name, metric in metricas.items():
    print(f'{name}: {metric(val_y, y_pred_grid): .4f}')


 Resultados del GridSearch
Tiempo de búsqueda:  3935.96 segundos
MAE:  0.0989
MSE:  0.0203
R2:  0.9799


**Resultados de RandomSearch**

In [21]:
print('\n Resultados del Random Search')
print(f'Tiempo de búsqueda: {random_time: .2f} segundos')
for name, metric in metricas.items():
    print(f'{name}: {metric(val_y, y_pred_random): .4f}')


 Resultados del Random Search
Tiempo de búsqueda:  511.43 segundos
MAE:  0.1282
MSE:  0.0338
R2:  0.9665


### 5.6 XGBRegressor

Instaciar el modelo

In [28]:
model = XGBRFRegressor(objective = 'reg:squarederror', random_state= 42, n_jobs = -1)

Definir los hiperparametros a probar

In [30]:
grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 6, 10],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

Entrenar bajo *GridSearchCV*

In [31]:
start_time = time.time()
grid_search = GridSearchCV(model, grid, cv = 3, scoring = 'r2', n_jobs = -1)
grid_search.fit(train_x, train_y)
grid_time = time.time() - start_time

Recuperar la mejor configuración

In [33]:
best_model_grid = grid_search.best_estimator_
best_model_grid

XGBRFRegressor(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bytree=0.8, device=None,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, feature_types=None, gamma=None,
               grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=0.2, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=10, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=50, n_jobs=-1,
               num_parallel_tree=None, objective='reg:squarederror',
               random_state=42, ...)

Entrenar bajo *RandomizedSearchCV*

In [32]:
start_time = time.time()
random_search = RandomizedSearchCV(model, param_distributions = grid, n_iter = 5, cv = 5, scoring = 'r2', n_jobs = -1, random_state = 42)
random_search.fit(train_x, train_y)
random_time = time.time() - start_time

Recuperar la mejor configuración

In [34]:
best_model_random = random_search.best_estimator_
best_model_random

XGBRFRegressor(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bytree=1.0, device=None,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, feature_types=None, gamma=None,
               grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=0.1, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=6, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=200, n_jobs=-1,
               num_parallel_tree=None, objective='reg:squarederror',
               random_state=42, ...)

Utilizar las mejores configuraciones sobres los datos de validación

In [35]:
y_pred_grid = best_model_grid.predict(val_x)
y_pred_random = best_model_random.predict(val_x)

**Resultados de GridSearch**

In [36]:
print('\n Resultados del GridSearch')
print(f'Tiempo de búsqueda: {grid_time: .2f} segundos')
for name, metric in metricas.items():
    print(f'{name}: {metric(val_y, y_pred_grid): .4f}')


 Resultados del GridSearch
Tiempo de búsqueda:  421.26 segundos
MAE:  0.6636
MSE:  0.7286
R2:  0.2775


**Resultados de RandomSearch**

In [37]:
print('\n Resultados del Random Search')
print(f'Tiempo de búsqueda: {random_time: .2f} segundos')
for name, metric in metricas.items():
    print(f'{name}: {metric(val_y, y_pred_random): .4f}')


 Resultados del Random Search
Tiempo de búsqueda:  20.36 segundos
MAE:  0.7415
MSE:  0.9085
R2:  0.0991


**Conclusiones iniciales:** El metodo de GridSearch se encuentra mejor posicionado, pese a tomar un tiempo mayor obtiene mejores resultados en general.

## 7. Conclusion

Agrupando los resultados de los modelos bajo el metodo de optimizacion *GridSearchCV* en la siguiente tabla

| Modelo | MSE | MAE | R2 | Time |
|-------|-----|-----|----|------|
|LinearRegression | 0.56 | 0.59 | 0.4 | 3.62 |
|Ridge | 0.56 | 0.59 | 0.4 | 3.18 |
|DecisionTreeRegressor | 0.24 | 0.18 | 0.81 | 77.08 |
|RandomForestRegressor | 0.16 | 0.06 | 0.93 | 578.09 |
|GradientBoostingRegressor | 0.09 | 0.02 | 0.97 | 3935.96 |
|XGBRegressor | 0.66 | 0.72 | 0.27 | 421.26 |

Se obtienen las conclusiones:

- GradientBoostingRegressor tiene el mejor desempeño en cuanto a resultados, aunque es el supera con creces los tiempos de computos de todos los ejercicios realizados.

- RandomForestRegressor toma el segundo lugar, teniendo solo una diferencia de 0.4 con el primer lugar. Pero compensando con un tiempo de computo bastante menor.

Si se toman todos los elementos en consideración, el modelo más adecuado seria RandomForestRegressor. Que cubre resultados optimos con eficiencia de tiempo, dado asi el mejor rendimiento bajo el metodo *GridSearchCV*.

La tabla con los resultados de *RandomizedSearchCV* se muestra a continuación

|Modelo | MSE | MAE | R2 | Time |
|-------|-----|-----|----|------|
|LinearRegression | 0.56 | 0.59 | 0.4 | 1.15 |
|Ridge | 0.56 | 0.59 | 0.4 | 2.34 |
|DecisionTreeRegressor | 0.24 | 0.18 | 0.81 | 12.29 |
|RandomForestRegressor | 0.17 | 0.06 | 0.93 | 680.20 |
|GradientBoostingRegressor | 0.12 | 0.02 | 0.96 | 511.43 |
|XGBRegressor | 0.74 | 0.90 | 0.09 | 20.36 |

Las concluciones que se obtienen son:

- En general, todos los modelos tienen tiempos de computos menores en relación al otro meotodo.

- GradientBoostingRegressor se posiciona como el mejor modelo en cuanto a resultados y rendimiento, en comparación al resto de modelo.

Como conclusion general, el modelo de GradientBoostingRegressor optimizado con *RandomizedSearchCV* es el que entrega el mejor rendimiento. Tiene una diferencia del 0.1 con la solución que obtiene el mejor resultado, pero su tiempo de computo es 7.69 veces menor que él. Por lo tanto es la solución más eficiente en terminos de resultados y tiempo.